In [7]:
import pandas as pd
import os
import glob

# --- [설정] 1. 데이터가 있는 폴더 ---
input_folder = "../sort_asc/"

# --- [설정] 2. 그룹화 및 합산할 "표준" 컬럼 이름 ---
# (data4도 이 이름으로 강제 변경할 것입니다)
group_column_name = '기준일자'
sum_column_name = '소비인구(명)'


try:
    # --- 1. 'sort_asc' 폴더의 모든 CSV 파일 경로 찾기 ---
    file_pattern = os.path.join(input_folder, "data*_sorted_asc.csv")
    all_csv_files = glob.glob(file_pattern)

    if not all_csv_files:
        print(f"[오류] '{input_folder}'에서 CSV 파일을 찾을 수 없습니다. 경로를 확인하세요.")
    else:
        print(f"총 {len(all_csv_files)}개의 파일을 찾았습니다:")
        all_csv_files.sort() # data1, data2 순서대로 정렬
        for f in all_csv_files:
            print(f"  - {f}")

    print("\n--- [작업 시작] 각 파일을 개별로 합산합니다. ---")

    # --- 2. 🔄 모든 CSV 파일을 *순회하며 개별 처리* ---
    for file_path in all_csv_files:

        base_name = os.path.basename(file_path)
        output_file_name = base_name.replace("_sorted_asc.csv", "_grouped.csv")

        print(f"\n--- [처리 중] '{base_name}' ---")

        # --- 3. 개별 파일 읽기 ---
        df = pd.read_csv(file_path)
        print(f"  > 파일 읽기 완료. (총 {len(df)}건)")

        # --- [ ★★★ 핵심 수정 ★★★ ] ---
        # data4의 컬럼명이 다른 경우, "표준" 컬럼명으로 강제 변경
        if "data4" in base_name:
            print("  > 'data4' 파일 감지. 8개 컬럼명 전체를 표준화합니다...")
            # 'de' -> '기준일자', 'cnsmr_popltn_co' -> '소비인구(명)'
            try:
                # [수정] data4의 8개 영문 컬럼을 표준 한글명으로 매핑
                df.rename(columns={
                    'gov_dn_cd': '행정동코드',
                    'brtc_nm': '시도명',
                    'signgu_nm': '시군구명',
                    'adstrd_nm': '행정동명',
                    'de': '기준일자',
                    'sex_se': '성별',
                    'year_se': '연령대',
                    'cnsmr_popltn_co': '소비인구(명)'
                }, inplace=True)
                print("  > 컬럼명 8개 변경 완료.")
            except Exception as e:
                print(f"  [오류] data4 컬럼명 변경 중 오류 발생: {e}")
                print("  > 이 파일을 건너뜁니다.")
                continue # 다음 파일로 넘어감
        # -------------------------------

        # --- 4. '소비인구(명)' 열을 숫자로 변환 ---
        # (이제 'sum_column_name' 변수 하나로 data1~4 모두 처리 가능)
        df[sum_column_name] = pd.to_numeric(df[sum_column_name], errors='coerce')
        df[sum_column_name] = df[sum_column_name].fillna(0)
        print("  > 숫자 변환 완료.")

        # --- 5. 📊 '기준일자'로 그룹화하고 '소비인구(명)' 합산 ---
        print(f"  > '{group_column_name}'별로 '{sum_column_name}'을(를) 합산 중...")

        grouped_series = df.groupby(group_column_name)[sum_column_name].sum()
        final_df = grouped_series.reset_index()

        final_df = final_df.sort_values(by=group_column_name, ascending=True)

        # --- 6. 💾 합산된 결과를 새 CSV 파일로 저장 ---
        print(f"  > 합산된 데이터를 '{output_file_name}' 파일로 저장 중...")
        final_df.to_csv(output_file_name, index=False, encoding='utf-8-sig')

        print(f"  [성공] '{output_file_name}' 파일 저장 완료!")
        print(f"    (총 {len(final_df)}건의 일자별 데이터)")


    print("\n\n--- [작업 완료] 모든 파일의 개별 합산을 완료했습니다. ---")

except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다. 경로를 확인하세요.")
except KeyError as e:
    print(f"[오류] '{e}' 컬럼을 찾을 수 없습니다. CSV 파일의 헤더를 확인하세요.")
except Exception as e:
    print(f"작업 중 알 수 없는 오류가 발생했습니다: {e}")

총 4개의 파일을 찾았습니다:
  - ../sort_asc\data1_sorted_asc.csv
  - ../sort_asc\data2_sorted_asc.csv
  - ../sort_asc\data3_sorted_asc.csv
  - ../sort_asc\data4_sorted_asc.csv

--- [작업 시작] 각 파일을 개별로 합산합니다. ---

--- [처리 중] 'data1_sorted_asc.csv' ---
  > 파일 읽기 완료. (총 1183696건)
  > 숫자 변환 완료.
  > '기준일자'별로 '소비인구(명)'을(를) 합산 중...
  > 합산된 데이터를 'data1_grouped.csv' 파일로 저장 중...
  [성공] 'data1_grouped.csv' 파일 저장 완료!
    (총 184건의 일자별 데이터)

--- [처리 중] 'data2_sorted_asc.csv' ---
  > 파일 읽기 완료. (총 590279건)
  > 숫자 변환 완료.
  > '기준일자'별로 '소비인구(명)'을(를) 합산 중...
  > 합산된 데이터를 'data2_grouped.csv' 파일로 저장 중...
  [성공] 'data2_grouped.csv' 파일 저장 완료!
    (총 92건의 일자별 데이터)

--- [처리 중] 'data3_sorted_asc.csv' ---
  > 파일 읽기 완료. (총 593417건)
  > 숫자 변환 완료.
  > '기준일자'별로 '소비인구(명)'을(를) 합산 중...
  > 합산된 데이터를 'data3_grouped.csv' 파일로 저장 중...
  [성공] 'data3_grouped.csv' 파일 저장 완료!
    (총 92건의 일자별 데이터)

--- [처리 중] 'data4_sorted_asc.csv' ---
  > 파일 읽기 완료. (총 594659건)
  > 'data4' 파일 감지. 8개 컬럼명 전체를 표준화합니다...
  > 컬럼명 8개 변경 완료.
  > 숫자 변환 완료.
  > '기준일자'별

In [12]:
import pandas as pd
import os

# --- [설정] 1. 데이터가 있는 폴더 ---
# (ipynb 파일 위치에서 한 단계 위로 올라가서 'sort_asc' 폴더로)
folder_path = "../sort_asc/"

# --- [설정] 2. 컬럼명을 변경할 대상 파일 ---
target_file_name = "data1_sorted_asc.csv"
target_file_path = os.path.join(folder_path, target_file_name)

# --- [설정] 3. 변경할 8개의 컬럼 매핑 ---
# ( data4의 영문명 -> data1~3의 한글명 )
final_column_order = [
    '행정동코드',
    '시도명',
    '시군구명',
    '행정동명',
    '기준일자',
    '성별',
    '연령대',
    '소비인구(명)'
]

try:
    # --- 1. 대상 파일 읽기 ---
    print(f"'{target_file_path}' 파일을 읽는 중...")
    df = pd.read_csv(target_file_path)


    # --- 3. [STEP 2] 컬럼 순서 재배치 ---
    print(f"'{target_file_name}' 파일의 컬럼 순서를 재배치합니다...")
    # [핵심] 정의된 final_column_order 리스트 순서대로 DataFrame을 재구성
    df = df[final_column_order]
    print("  > 컬럼 순서 재배치 완료.")

    # --- 4. [STEP 3] 변경된 DataFrame을 *원본 파일 경로에 덮어쓰기* ---
    print(f"변경된 내용으로 '{target_file_path}' 파일을 덮어쓰는 중...")

    df.to_csv(target_file_path, index=False, encoding='utf-8-sig')

    print(f"[성공] '{target_file_path}' 파일의 컬럼명과 순서가 영구적으로 변경되었습니다.")

    # --- 5. 변경 결과 확인 ---
    print("\n--- 변경된 파일의 상위 5개 행 (최종 순서) ---")
    print(df.head())

except FileNotFoundError:
    print(f"[오류] '{target_file_path}' 파일을 찾을 수 없습니다. 경로와 파일 이름을 확인하세요.")
except KeyError as e:
    print(f"[오류] {e} 컬럼을 찾을 수 없습니다.")
    print("  > 'column_mapping' (이름 변경) 또는 'final_column_order' (순서 변경)에 오타가 없는지 확인하세요.")
except Exception as e:
    print(f"작업 중 알 수 없는 오류가 발생했습니다: {e}")

'../sort_asc/data1_sorted_asc.csv' 파일을 읽는 중...
'data1_sorted_asc.csv' 파일의 컬럼 순서를 재배치합니다...
  > 컬럼 순서 재배치 완료.
변경된 내용으로 '../sort_asc/data1_sorted_asc.csv' 파일을 덮어쓰는 중...
[성공] '../sort_asc/data1_sorted_asc.csv' 파일의 컬럼명과 순서가 영구적으로 변경되었습니다.

--- 변경된 파일의 상위 5개 행 (최종 순서) ---
        행정동코드    시도명 시군구명  행정동명      기준일자 성별 연령대     소비인구(명)
0  1165053000  서울특별시  서초구  서초3동  20230501  F  25  144.027005
1  1111064000  서울특별시  종로구   이화동  20230501  F  15   64.012002
2  1111064000  서울특별시  종로구   이화동  20230501  F  20  152.028506
3  1111064000  서울특별시  종로구   이화동  20230501  F  25  208.039008
4  1111064000  서울특별시  종로구   이화동  20230501  F  30  136.025505
